In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/interactive-sales-dashboard-data/US Superstore data.xls


In [2]:
# ============================================
# STEP 1: IMPORT LIBRARIES
# ============================================
import pandas as pd
import plotly.express as px
import ipywidgets as widgets
from IPython.display import display

# ============================================
# STEP 2: LOAD DATA
# ============================================
file_path = "/kaggle/input/interactive-sales-dashboard-data/US Superstore data.xls"
df = pd.read_excel(file_path)

# ============================================
# STEP 3: PREPROCESS DATA
# ============================================
df.rename(columns={"Order Date": "Order_Date"}, inplace=True)
df["Order_Date"] = pd.to_datetime(df["Order_Date"])
df["Year"] = df["Order_Date"].dt.year
df["Month"] = df["Order_Date"].dt.month_name()

# ============================================
# STEP 4: WIDGET FILTERS
# ============================================
region_filter = widgets.SelectMultiple(
    options=sorted(df["Region"].unique()),
    description="Region",
    layout={'width': '300px'}
)

category_filter = widgets.SelectMultiple(
    options=sorted(df["Category"].unique()),
    description="Category",
    layout={'width': '300px'}
)

year_filter = widgets.SelectMultiple(
    options=sorted(df["Year"].unique()),
    description="Year",
    layout={'width': '300px'}
)

# ============================================
# STEP 5: DASHBOARD FUNCTION
# ============================================
def update_dashboard(*args):

    dff = df.copy()

    # Filters
    if region_filter.value:
        dff = dff[dff["Region"].isin(region_filter.value)]

    if category_filter.value:
        dff = dff[dff["Category"].isin(category_filter.value)]

    if year_filter.value:
        dff = dff[dff["Year"].isin(year_filter.value)]

    # KPIs
    total_sales = dff["Sales"].sum()
    total_profit = dff["Profit"].sum()
    total_orders = dff["Order ID"].nunique()

    print("===== KPIs =====")
    print(f"💰 Total Sales : ${total_sales:,.2f}")
    print(f"📈 Total Profit: ${total_profit:,.2f}")
    print(f"🛒 Total Orders: {total_orders:,}")
    print("=================\n")

    # --------------------------------------------
    # Chart 1 — Revenue by Region
    # --------------------------------------------
    fig1 = px.bar(
        dff.groupby("Region")["Sales"].sum().reset_index(),
        x="Region",
        y="Sales",
        title="Revenue by Region"
    )
    fig1.show()

    # --------------------------------------------
    # Chart 2 — Sales Over Time
    # --------------------------------------------
    fig2 = px.line(
        dff.groupby(["Year", "Month"])["Sales"].sum().reset_index(),
        x="Month",
        y="Sales",
        color="Year",
        title="Sales Trend (Monthly)"
    )
    fig2.show()

    # --------------------------------------------
    # Chart 3 — Top 10 Products
    # --------------------------------------------
    fig3 = px.bar(
        dff.groupby("Product Name")["Sales"].sum()
        .sort_values(ascending=False).head(10).reset_index(),
        x="Product Name",
        y="Sales",
        title="Top 10 Products by Sales"
    )
    fig3.show()

# ============================================
# STEP 6: LINK FILTERS TO FUNCTION
# ============================================
region_filter.observe(update_dashboard, names='value')
category_filter.observe(update_dashboard, names='value')
year_filter.observe(update_dashboard, names='value')

# ============================================
# DISPLAY FILTERS
# ============================================
print("### 🔽 Use these filters to update dashboard:")
display(region_filter, category_filter, year_filter)

# Initial dashboard load
update_dashboard()


### 🔽 Use these filters to update dashboard:


SelectMultiple(description='Region', layout=Layout(width='300px'), options=('Central', 'East', 'South', 'West'…

SelectMultiple(description='Category', layout=Layout(width='300px'), options=('Furniture', 'Office Supplies', …

SelectMultiple(description='Year', layout=Layout(width='300px'), options=(2014, 2015, 2016, 2017), value=())

===== KPIs =====
💰 Total Sales : $2,297,200.86
📈 Total Profit: $286,397.02
🛒 Total Orders: 5,009

